In [1]:
import keras

2024-02-23 09:10:52.364136: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 09:10:54.599875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 09:10:54.599962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 09:10:54.962941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 09:10:55.689410: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-23 09:10:55.690389: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
import keras
from keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)  # Load without the top classification layers

# Get summary of the model's architecture
model.summary()

# Get information about the layers
for layer in model.layers:
    print(f"Layer Name: {layer.name}, Input Shape: {layer.input_shape}, Output Shape: {layer.output_shape}")


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [11]:
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load an image and pre-process it
img_path = "MCL750s.jpg"  # Replace with your image path
img = load_img(img_path, target_size=(224, 224))  # Resize to match VGG16 input size
x = img_to_array(img)
x = np.expand_dims(x, axis=0)  # Add batch dimension
x = preprocess_input(x)

# Predict the class using VGG16 (requires loading top layers)
model = VGG16(weights='imagenet')  # Load with top layers
predictions = model.predict(x)
predicted_class = decode_predictions(predictions, top=4)[0][0]
print(f"Predicted class: {predicted_class}")

1/1 [==============================] - 0s 296ms/step
Predicted class: ('n04285008', 'sports_car', 0.9579621)


In [12]:
# Freeze convolutional base layers to prevent them from being updated during training
for layer in model.layers[:15]:
    layer.trainable = False

# Use the model as a feature extractor
feature_output = model.predict(x)  # Extract features from the convolutional base

# Use the extracted features for downstream tasks (e.g., classification, clustering)
# ...


1/1 [==============================] - 0s 210ms/step


In [13]:
from keras.layers import Dense, Flatten

# Add new layers on top of the frozen base
num_classes = 1000  # Adjust based on your classification task
new_model = keras.Sequential([
    model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Print the total number of parameters
print(f"Total number of parameters: {new_model.count_params()}")


Total number of parameters: 138870800


In [14]:
# Freeze only the convolutional base layers, allowing the rest to be trained
for layer in new_model.layers[:15]:
    layer.trainable = False


In [15]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1000)              138357544 
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 256)               256256    
                                                                 
 dense_1 (Dense)             (None, 1000)              257000    
                                                                 
Total params: 138870800 (529.75 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 138870800 (529.75 MB)
_________________________________________________________________
